<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# mcdxa Package Tutorial

This notebook demonstrates how to use the **mcdxa** package for option pricing.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from mcdxa.models import BSM, Heston, Merton, Bates
from mcdxa.payoffs import CallPayoff, PutPayoff, AsianCallPayoff, CustomPayoff
from mcdxa.pricers.european import EuropeanPricer
from mcdxa.pricers.american import AmericanBinomialPricer, LongstaffSchwartzPricer
from mcdxa.analytics import bsm_price

## Payoff Examples


In [ ]:
K = 100
spots = [80, 100, 120]
call = CallPayoff(K)
put = PutPayoff(K)
print('Call payoff:', call(spots))
print('Put payoff :', put(spots))

## Custom Payoff


In [ ]:
sqrt_call = CustomPayoff(lambda s: np.maximum(np.sqrt(s) - 9, 0))
spots = [81, 100, 121]
print('Custom sqrt call payoff:', sqrt_call(spots))

## Path-dependent Payoff (Asian)


In [ ]:
paths = np.array([[90, 110, 130], [120, 100, 80]])
asian_call = AsianCallPayoff(100)
print('Asian call payoff on sample paths:', asian_call(paths))

## Simulate and Plot BSM Paths


In [ ]:
model = BSM(r=0.05, sigma=0.2)
paths = model.simulate(100, 1, n_paths=5, n_steps=50)
plt.figure(figsize=(8,4))
for i, path in enumerate(paths):
    plt.plot(path, label=f'Path {i}')
plt.legend(); plt.title('BSM Sample Paths'); plt.xlabel('Step'); plt.ylabel('Price')
plt.show()

## European Option Pricing via Monte Carlo


In [ ]:
model = BSM(r=0.05, sigma=0.2)
payoff = CallPayoff(100)
pricer = EuropeanPricer(model, payoff, n_paths=5000, n_steps=50, seed=42)
price_mc, stderr = pricer.price(100, 1, 0.05)
price_bs = bsm_price(100, 100, 1, 0.05, 0.2, option_type='call')
print(f'MC price: {price_mc:.4f} ± {stderr:.4f}, BSM price: {price_bs:.4f}')

## American Option Pricing


In [ ]:
amer = AmericanBinomialPricer(model, CallPayoff(100), n_steps=100)
price_amer = amer.price(100, 1, 0.05)
print('CRR American call price:', price_amer)
lsm = LongstaffSchwartzPricer(model, PutPayoff(100), n_paths=5000, n_steps=50, seed=42)
price_lsm, stderr_lsm = lsm.price(90, 1, 0.05)
print(f'LSM American put price: {price_lsm:.4f} ± {stderr_lsm:.4f}')

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>